In [ ]:
%pip install -r requirements.txt

In [1]:
#use logger
import logging
import time
import utils.config as config
import utils.io_utils as io_utils
if __name__ == "__main__":
    try:
        #logging.basicConfig(level=logging.DEBUG)
        start_time = time.time()
        io_utils.extract_EaglePack_and_process(config.input_path)
        end_time = time.time()
        print(f"Total processing time: {end_time - start_time:.2f} seconds")
    finally:
        if config.use_LLM:
            config.llm_processor.stop()
    print("Processing completed")

Not using LLM


DEBUG:asyncio:Using selector: SelectSelector

Total Images Processed:   0%|          | 0/5336 [00:00<?, ?it/s]

g:\anaconda3\envs\aes\lib\site-packages\tqdm\std.py:636: TqdmWarning: clamping frac to range [0, 1]
Total Images Processed:   0%|          | 1/5336 [00:00<24:39,  3.61it/s]DEBUG:utils.upscale_API:Batch tensor shape: torch.Size([1, 3, 694, 694])

Total Images Processed:   5%|▍         | 253/5336 [00:01<00:33, 153.90it/s]DEBUG:utils.upscale_API:Output batch tensor shape: torch.Size([1, 3, 1388, 1388])
INFO:utils.upscale_API:Saved upscaled image to: P:\datasets\stickers\base\img_000.png

Total Images Processed:   5%|▌         | 277/5336 [00:02<00:32, 154.26it/s]DEBUG:utils.upscale_API:Batch tensor shape: torch.Size([1, 3, 694, 694])

Total Images Processed:   8%|▊         | 450/5336 [00:03<00:31, 153.27it/s]

Total Images Processed:   9%|▉         | 470/5336 [00:03<00:33, 146.82it/s]pscale_API:Output batch tensor shape: torch.Size([1, 3, 1388, 1388])INFO:utils.upscale_API:Saved 